# Ground Plane Fitting and Scan Line Run for 3D LiDAR

Ground Plane Fitting (GPF) and Naive Baseline for 3D LiDAR Segmentation

This notebook implements ground segmentation using the Ground Plane Fitting (GPF) algorithm 
proposed in:

"Fast Segmentation of 3D Point Clouds: A Paradigm on LiDAR Data for Autonomous Vehicle Applications"
by D. Zermas, I. Izzat, and N. Papanikolopoulos, 2017.

The implementation also includes a naive baseline method for comparison, as well as 
basic clustering and visualization tools.

# Imports

In [1]:
import numpy as np
from pypcd import pypcd  # Use local pypcd version (Python 3 compatible, no lzf support)
from sklearn.decomposition import PCA  # Used for plane fitting
from sklearn.neighbors import NearestNeighbors  # Used for spatial clustering
import pptk  # To install: pip install pptk
import os
from collections import defaultdict
from pathlib import Path
import numpy as np
import yaml
# Note: Ubuntu users may need to fix libz.so.1 symlink issue (see: https://github.com/heremaps/pptk/issues/3)

# Hyperparameters

In [2]:
# # Number of lowest points to use for initial ground seed estimation (LPR)
# N_LPR = 1000

# # Number of iterations for ground plane refinement
# N_ITERATIONS = 6

# # Height threshold above the LPR to select initial seed points
# TH_SEEDS = 0.4

# # Distance threshold to classify a point as ground
# TH_DISTANCE_FROM_PLANE = 0.2

# RADIUS_FOR_NEAREST_NEIGHBORS = 0.5

# MIN_POINTS_FOR_CLUSTER = 1


# Naive Baseline Method

In [3]:
def naive_ground_extractor(point_cloud, num_lowest_points):
    """
    Naive ground extraction method (baseline).
    
    This simple method selects the points with the lowest Z values 
    and assumes they belong to the ground surface. It does not model 
    the ground plane and is used as a baseline for comparison against 
    more robust algorithms like Ground Plane Fitting (GPF).
    
    Args:
        point_cloud (np.ndarray): N x 3 or N x D array of point cloud data.
        num_lowest_points (int): Number of points with lowest Z values to classify as ground.
    
    Returns:
        ground_indices (np.ndarray): Indices of the selected ground points.
    """

    # Step 1: Sort points by Z axis (height)
    sorted_indices = np.argsort(point_cloud[:, 2])

    # Step 2: Take the first N indices as ground
    ground_indices = sorted_indices[:num_lowest_points]

    return ground_indices

# Ground Plane Fitting (GPF)

In [4]:
def gpf_extract_initial_seeds_ids(point_cloud, number_of_lowest_points=1000, threshold_seeds=0.4):
    """
    Extract initial seed points for ground plane estimation (GPF).
    
    Args:
        point_cloud (np.ndarray): N x 3 array of points (x, y, z).
        number_of_lowest_points (int): number of lowest Z points to average as LPR.
        threshold_seeds (float): threshold to select seeds close to LPR height.
    
    Returns:
        seeds_ids (np.ndarray): indices of points selected as initial seeds.
    """

    # Step 1: Sort the point cloud by Z axis (height)
    sorted_indices = np.argsort(point_cloud[:, 2])          # Get indices sorted by height
    sorted_points = point_cloud[sorted_indices]             # Apply sorting

    # Step 2: Compute LPR (Lowest Point Representative)
    LPR_height = np.mean(sorted_points[:number_of_lowest_points, 2])

    # Step 3: Select points that are within threshold distance from LPR
    seed_mask = sorted_points[:, 2] < (LPR_height + threshold_seeds)
    seeds_ids = sorted_indices[seed_mask]

    return seeds_ids

In [5]:
# def gpf_estimate_plane(seed_points):
#     """
#     Estimate ground plane from seed points using PCA.
    
#     Args:
#         seed_points (np.ndarray): N x 3 or N x D array of selected ground seed points.
    
#     Returns:
#         normal_vector (np.ndarray): Unit normal vector [a, b, c] of the estimated plane.
#         d (float): Plane offset in the equation ax + by + cz + d = 0
#     """

#     # Step 1: Compute the mean position of the seed points
#     mean_point = np.mean(seed_points, axis=0)

#     # Step 2: Center the point cloud around the origin
#     centered = seed_points - mean_point

#     # Step 3: Use PCA to estimate the plane's normal vector
#     pca = PCA(n_components=3)
#     pca.fit(centered)
#     normal_vector = pca.components_[-1]  # Last component corresponds to the direction with least variance

#     # Step 4: Compute the 'd' value using plane equation: n·x + d = 0 → d = -n·x̄
#     d = -np.dot(normal_vector[:3], mean_point[:3])

#     return normal_vector[:3], d

In [6]:
def gpf_estimate_plane(points):
    # pts = np.array(points)
    mean = np.mean(points, axis=0)
    centered = points - mean

    cov = np.cov(centered.T)
    _, _, vh = np.linalg.svd(cov)
    normal = vh[-1]

    d = -np.dot(normal, mean)
    return (normal, d)

In [7]:
def gpf_refinement(point_cloud, number_of_lowest_points=1000, threshold_seeds=0.4, plane_distance_threshold=0.2, num_iterations=5):
    """
    Iteratively refine the ground plane estimation using seed points and distance threshold.
    
    Args:
        point_cloud (np.ndarray): N x 3 or N x D array of input point cloud.
        initial_seed_indices (np.ndarray): Indices of initial ground seed points.
        distance_threshold (float): Max allowed point-to-plane distance to be considered ground.
        num_iterations (int): Number of iterations for refinement loop.
    
    Returns:
        ground_indices (np.ndarray): Indices of points classified as ground.
        non_ground_indices (np.ndarray): Indices of points classified as non-ground.
    """

    xyz_point_cloud = point_cloud[:, :3]
    seed_indices = gpf_extract_initial_seeds_ids(xyz_point_cloud, number_of_lowest_points, threshold_seeds)

    for _ in range(num_iterations):
        # Step 1: Estimate plane from current seed points
        normal_vector, d = gpf_estimate_plane(xyz_point_cloud[seed_indices])

        # Step 2: Compute point-to-plane distances
        numerator = np.abs(np.dot(xyz_point_cloud, normal_vector) + d)
        denominator = np.linalg.norm(normal_vector)
        distances = numerator / denominator

        # Step 3: Classify points as ground or non-ground
        is_ground = distances < plane_distance_threshold

        # Step 4: Update seed indices with the new ground points
        seed_indices = np.where(is_ground)[0]

    # Final classification after last iteration
    ground_indices = np.where(is_ground)[0]
    point_cloud[ground_indices, 4] = 9

    return point_cloud, normal_vector, d

# Scan Line Run (SLR)

In [8]:
def cluster_connected_components(points, eps=0.5, min_samples=1):
    """
    Cluster points into connected components using radius-based neighborhood and Union-Find.

    This method groups spatially connected points into clusters. It is a simplified version 
    of DBSCAN without density-based core points — it only relies on spatial proximity and 
    minimum cluster size filtering.

    Args:
        points (np.ndarray): N x 3 (or N x D) array of point coordinates.
        eps (float): Maximum distance between neighbors (radius in meters).
        min_samples (int): Minimum number of points to form a valid cluster.

    Returns:
        final_labels (np.ndarray): Cluster labels for each point (-1 for outliers).
    """
    # Step 1: Build neighborhood graph using radius search
    neighbors = NearestNeighbors(radius=eps).fit(points)
    adjacency = neighbors.radius_neighbors_graph(points, mode='connectivity').tocoo()

    # Step 2: Union-Find (Disjoint Set) initialization
    parent = np.arange(len(points))

    def find(i):
        # Path compression
        while parent[i] != i:
            parent[i] = parent[parent[i]]
            i = parent[i]
        return i

    def union(i, j):
        # Union by parent
        root_i, root_j = find(i), find(j)
        if root_i != root_j:
            parent[root_i] = root_j

    # Step 3: Connect all neighbors
    for i, j in zip(adjacency.row, adjacency.col):
        if i != j:
            union(i, j)

    # Step 4: Assign cluster labels
    labels = np.array([find(i) for i in range(len(points))])

    # Step 5: Filter small clusters (outliers)
    unique_labels, counts = np.unique(labels, return_counts=True)
    valid_labels = unique_labels[counts >= min_samples]
    mask = np.isin(labels, valid_labels)

    # Step 6: Remap valid clusters to consecutive indices, mark outliers as -1
    final_labels = -np.ones_like(labels)
    final_labels[mask] = np.searchsorted(valid_labels, labels[mask])
    return final_labels

In [9]:
def process_non_ground_clusters(point_cloud, eps=0.5, min_pts=1):
    """
    Apply spatial clustering to points not classified as ground.

    This function filters out ground points (using provided indices),
    and applies connected components clustering to the remaining non-ground points.

    Args:
        point_cloud (np.ndarray): N x 4 array with [x, y, z, class] per point.
        ground_indices (np.ndarray): Indices of points classified as ground.

    Returns:
        cluster_labels (np.ndarray): Cluster label for each non-ground point (-1 for noise).
        non_ground_indices (np.ndarray): Indices of the non-ground points in the original array.
    """

    # Step 1: Identify non-ground point indices
    non_ground_indices = np.where(point_cloud[:, 4] != 9)[0]

    # Step 2: Extract non-ground points for clustering
    non_ground_points = point_cloud[non_ground_indices, :3]  # Use only XYZ for clustering

    # Step 3: Apply spatial clustering to non-ground points
    cluster_labels = cluster_connected_components(non_ground_points, eps=eps, min_samples=min_pts)

    point_cloud[non_ground_indices, 4] = cluster_labels

    return point_cloud


# Visualizer for Point Clouds

In [10]:
def visualize_clusters(
    point_cloud,
    non_ground_indices=None,
    cluster_labels=None,
    point_size=0.03,
    show_ground=True,
    show_clusters=True,
    show_unlabeled=True,
    show_plane=False,
    show_true_label=False  # New parameter to toggle label source
):
    """
    Visualize a point cloud with consistent class colors using pptk.

    Args:
        point_cloud (np.ndarray): N x 7 array with [x, y, z, reflectance, gt_label, pred_label, ...].
        non_ground_indices (np.ndarray): Indices of non-ground points.
        cluster_labels (np.ndarray): Cluster labels for non-ground points (-1 = noise).
        point_size (float): Size of points in viewer.
        show_ground (bool): Show ground points (label == 1).
        show_clusters (bool): Show clustered non-ground points.
        show_unlabeled (bool): Show points with label == 0.
        show_true_label (bool): Use ground truth labels (col 5) or predicted (col 6).

    Returns:
        None
    """
    # Copy the point cloud to avoid modifying original
    pc = np.copy(point_cloud)

    # Column index to use for visualization
    label_col = 3 if show_true_label else 4

    # Assign predicted cluster labels if applicable
    if cluster_labels is not None and non_ground_indices is not None and show_clusters:
        for cluster_id in np.unique(cluster_labels):
            if cluster_id == -1:
                continue  # skip noise
            cluster_point_ids = non_ground_indices[cluster_labels == cluster_id]
            pc[cluster_point_ids, label_col] = cluster_id

    # Create mask for points to visualize
    show_mask = np.full(pc.shape[0], False)
    if show_plane:
        show_mask |= (pc[:, label_col] == -1)
    if show_unlabeled:
        show_mask |= (pc[:, label_col] == 0)
    if show_ground:
        show_mask |= (pc[:, label_col] == 1)
    if show_clusters:
        show_mask |= (pc[:, label_col] > 1)

    # Extract visible points and labels
    xyz = pc[show_mask, :3]
    class_labels = pc[show_mask, label_col].astype(int)

    # Fixed color palette for known classes
    fixed_colors = {
        -1: [255, 255, 255],
        0: [0, 0, 0],
        1: [245, 150, 100],
        2: [245, 230, 100],
        3: [150, 60, 30],
        4: [180, 30, 80],
        5: [250, 80, 100],
        6: [30, 30, 255],
        7: [200, 40, 255],
        8: [90, 30, 150],
        9: [255, 0, 255],
        10: [255, 150, 255],
        11: [75, 0, 75],
        12: [75, 0, 175],
        13: [0, 200, 255],
        14: [50, 120, 255],
        15: [0, 175, 0],
        16: [0, 60, 135],
        17: [80, 240, 150],
        18: [150, 240, 255],
        19: [0, 0, 255],
    }
    # Convert BGR to RGB and normalize
    fixed_colors_rgb_normalized = {
        label: [bgr[2]/255.0, bgr[1]/255.0, bgr[0]/255.0]  # [R, G, B]
        for label, bgr in fixed_colors.items()
    }

    # Re-map visible class labels to continuous indices for pptk
    unique_labels = np.unique(class_labels)
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    mapped_labels = np.array([label_to_index[label] for label in class_labels])

    # Build color map
    color_map = []
    for label in unique_labels:
        if label in fixed_colors_rgb_normalized:
            color_map.append(fixed_colors_rgb_normalized[label])
        else:
            np.random.seed(label)  # keep color stable per class
            color_map.append(np.random.rand(3))  # fallback

    # Launch viewer
    viewer = pptk.viewer(xyz, mapped_labels)
    viewer.set(point_size=point_size)
    viewer.color_map(color_map)

## Generate plane

In [11]:
def generate_plane_points(normal, d, center, size=10, resolution=0.5):
    """
    Gera pontos em um plano com base na equação ax + by + cz + d = 0

    Args:
        normal (np.ndarray): vetor normal (a, b, c)
        d (float): distância do plano
        center (np.ndarray): centro do plano (x, y, z)
        size (float): tamanho do plano
        resolution (float): espaçamento entre os pontos

    Returns:
        np.ndarray: Mx3 array de pontos do plano
    """
    a = normal[0]
    b = normal[1]
    c = normal[2]
    # Gera grid ao redor do centro
    x_vals = np.arange(center[0] - size, center[0] + size, resolution)
    y_vals = np.arange(center[1] - size, center[1] + size, resolution)
    xx, yy = np.meshgrid(x_vals, y_vals)
    
    # Equação do plano: ax + by + cz + d = 0  ->  z = (-d - ax - by)/c
    zz = (-d - a * xx - b * yy) / c

    xyz = np.stack((xx, yy, zz), axis=-1).reshape(-1, 3)
    l1 = np.full((xyz.shape[0], 1), -1, dtype=np.float32)
    l2 = np.full((xyz.shape[0], 1), -1, dtype=np.float32)
    # alterei aqui -----------------------------------
    # alterei aqui -----------------------------------
    # alterei aqui -----------------------------------
    l3 = np.full((xyz.shape[0], 1), -1, dtype=np.float32)

    plane_5d = np.hstack((xyz, l1, l2, l3))
    # plane_5d = np.hstack((xyz, l1, l2))
    return plane_5d

# Dataset

In [12]:
class Dataset:
    def __init__(self, data_path, split='train'):
        self.data_path = Path(data_path)
        self.yaml_path = self.data_path/'semantic-kitti.yaml'
        self.velodynes_path = self.data_path/'data_odometry_velodyne/dataset/sequences'
        self.labels_path = self.data_path/'data_odometry_labels/dataset/sequences'

        self.is_test = (split == 'test')

        # Load YAML metadata
        with open(self.yaml_path, 'r') as file:
            metadata = yaml.safe_load(file)

        self.sequences = metadata['split'][split]
        self.learning_map = metadata['learning_map']

        max_key = max(self.learning_map.keys())

        self.learning_map_np = np.zeros((max_key + 1,), dtype=np.uint32)
        for k, v in self.learning_map.items():
            self.learning_map_np[k] = v

        # List all frames
        self.frame_paths = []
        for sequence in self.sequences:
            sequence = f"{int(sequence):02d}"
            velo_files = sorted((self.velodynes_path/sequence/'velodyne').glob('*.bin'))
            for file in velo_files:
                self.frame_paths.append((sequence, file.stem))

    def __get_scan_ids_from_order__(self, point_cloud, n_scan=64):
        total_points = point_cloud.shape[0]
        scan_ids = np.arange(total_points) // (total_points // n_scan)
        return scan_ids

    def __len__(self):
        return len(self.frame_paths)

    def __getitem__(self, idx):
        seq, frame_id = self.frame_paths[idx]

        velodyne_path = self.velodynes_path/seq/'velodyne'/f"{frame_id}.bin"
        label_path = self.labels_path/seq/'labels'/f"{frame_id}.label"

        # Load point cloud
        with open(velodyne_path, 'rb') as file:
            point_cloud = np.fromfile(file, dtype=np.float32).reshape(-1, 4)[:, :3]

        # Load labels
        label = None
        mask = None
        if not self.is_test and label_path.exists():
            with open(label_path, 'rb') as file:
                label = np.fromfile(file, dtype=np.uint32) & 0xFFFF
            label = self.learning_map_np[label]
            mask = label != 0
        else:
            label = np.zeros(point_cloud.shape[0], dtype=np.uint32)
            mask = np.ones(point_cloud.shape[0], dtype=bool)

        item = {
            'point_cloud': point_cloud,
            'label': label,
            'mask': mask.astype(bool)
        }

        # alteracao aqui ------------------------------------------
        # alteracao aqui ------------------------------------------
        # alteracao aqui ------------------------------------------
        scan_ids = self.__get_scan_ids_from_order__(point_cloud, n_scan=64)
        scan_ids = scan_ids.reshape(-1, 1)
        point_cloud_with_label = np.hstack((point_cloud, label.reshape(-1, 1), np.zeros((point_cloud.shape[0], 1)), scan_ids))

        # alteracao aqui ------------------------------------------
        # descomentar tudo abaixo ------------------------------------------
        # ---------------------------------------------------------------
        # point_cloud_with_label = np.hstack((point_cloud, label.reshape(-1, 1)))
        # adicionando a coluna para ser a coluna de labels gerada pelo algoritmo
        # zeros_column = np.zeros((point_cloud.shape[0], 1), dtype=np.float32)
        # point_cloud_with_label = np.hstack((point_cloud_with_label, zeros_column))

        return point_cloud_with_label, item

# Main

In [13]:
def main(pcd_file):
    """
    Process a single point cloud file using GPF + optional clustering.
    Visualizes result using pptk.
    """

    # Load PCD file
    point_cloud_from_path = pypcd.PointCloud.from_path(pcd_file)

    # Create Nx4 array: x, y, z, class
    point_cloud = np.stack((
        point_cloud_from_path.pc_data['x'],
        point_cloud_from_path.pc_data['y'],
        point_cloud_from_path.pc_data['z'],
        np.zeros((point_cloud_from_path.pc_data.shape[0])), # default class 0
        np.zeros((point_cloud_from_path.pc_data.shape[0])),
    ), axis=1)

    # Step 1: Extract ground seeds using GPF

    # Step 2: Refine ground segmentation
    point_cloud, normal, d = gpf_refinement(point_cloud, 
                                        number_of_lowest_points=N_LPR, 
                                        threshold_seeds=TH_SEEDS, 
                                        plane_distance_threshold=TH_DISTANCE_FROM_PLANE,
                                        num_iterations=N_ITERATIONS)

    # Mark ground points with class = 1
    # point_cloud[ground_ids, 3] = 1

    # Step 3: Cluster non-ground points (SLR-style)
    # pc, cluster_labels, non_ground_ids = process_non_ground_clusters(point_cloud, non_ground_ids, eps=0.5, min_pts=1)

    center = point_cloud[:, :3].mean(axis=0)
    plane = generate_plane_points(normal, d, center)
    point_cloud = np.vstack((point_cloud, plane))

    # Step 4: Visualize result
    visualize_clusters(point_cloud, show_plane=True)

# point_cloud = process_non_ground_clusters(point_cloud, eps=0.5, min_pts=1)

In [14]:
# visualize_clusters(point_cloud, non_ground_ids, cluster_labels)
# cluster_labels.shape, cluster_labels

# Run Example

In [15]:
# Number of lowest points to use for initial ground seed estimation (LPR)
N_LPR = 2000

# Number of iterations for ground plane refinement
N_ITERATIONS = 5

# Height threshold above the LPR to select initial seed points
TH_SEEDS = 0.4

# Distance threshold to classify a point as ground
TH_DISTANCE_FROM_PLANE = 0.2

RADIUS_FOR_NEAREST_NEIGHBORS = 0.5

MIN_POINTS_FOR_CLUSTER = 1

In [16]:
dataset = Dataset('../datasets/semantic-kitti-data')
point_cloud, item = dataset[0]

point_cloud, normal, d = gpf_refinement(point_cloud, 
                                        number_of_lowest_points=N_LPR, 
                                        threshold_seeds=TH_SEEDS, 
                                        plane_distance_threshold=TH_DISTANCE_FROM_PLANE,
                                        num_iterations=N_ITERATIONS)


point_cloud = process_non_ground_clusters(point_cloud, eps=RADIUS_FOR_NEAREST_NEIGHBORS, min_pts=MIN_POINTS_FOR_CLUSTER)

center = point_cloud[:, :3].mean(axis=0)
plane = generate_plane_points(normal, d, center)
point_cloud = np.vstack((point_cloud, plane))

visualize_clusters(point_cloud, show_plane=True)

np.save('frame_segmentado_slr_diferente.npy', point_cloud)

In [17]:
# pcd_file = './pointclouds/1504941060.199916000.pcd'

# point_cloud_from_path = pypcd.PointCloud.from_path(pcd_file)

# point_cloud = np.stack((point_cloud_from_path.pc_data['x'], 
#                         point_cloud_from_path.pc_data['y'], 
#                         point_cloud_from_path.pc_data['z'], 
#                         np.zeros((point_cloud_from_path.pc_data.shape[0])),
#                         np.zeros((point_cloud_from_path.pc_data.shape[0])),
#                         np.zeros((point_cloud_from_path.pc_data.shape[0])),
#                         ), 
#                         axis=1)

# point_cloud, normal, d = gpf_refinement(point_cloud, 
#                                         number_of_lowest_points=N_LPR, 
#                                         threshold_seeds=TH_SEEDS, 
#                                         plane_distance_threshold=TH_DISTANCE_FROM_PLANE,
#                                         num_iterations=N_ITERATIONS)

# point_cloud = process_non_ground_clusters(point_cloud, eps=0.5, min_pts=1)

# center = point_cloud[:, :3].mean(axis=0)
# plane = generate_plane_points(normal, d, center)
# point_cloud = np.vstack((point_cloud, plane))

# visualize_clusters(point_cloud, show_plane=True)

# Testes

In [18]:
def ground_plane_fitting(point_cloud, N_ITERATIONS, NLPR, Thseeds, Thdist):
    xyz_point_cloud = point_cloud[:, :3]
    segments = split_into_segments(xyz_point_cloud, N_ITERATIONS)
    is_ground_list = []  # Lista que vai juntar os pedaços da máscara

    for seg in segments:
        is_ground_mask, plane_model = gpf_single_segment(seg, NLPR, Thseeds, Thdist)
        is_ground_list.append(is_ground_mask)
    
    is_ground = np.concatenate(is_ground_list)
    ground_indices = np.where(is_ground)[0]
    print('is:', is_ground.shape, is_ground)
    print('pc:', ground_indices.shape, ground_indices, ground_indices.max())
    point_cloud[ground_indices, 4] = 9
    
    return point_cloud, plane_model

def gpf_single_segment(point_cloud, NLPR, Thseeds, Thdist):
    seeds = extract_initial_seeds(point_cloud, NLPR, Thseeds)
    plane_model = estimate_plane(seeds)
    normal_vector, d = plane_model

    numerator = np.abs(np.dot(point_cloud, normal_vector) + d)
    denominator = np.linalg.norm(normal_vector)
    distances = numerator / denominator

    # Step 3: Classify points as ground or non-ground
    is_ground = distances < Thdist 

    return is_ground, plane_model

def extract_initial_seeds(points, NLPR, Thseeds):
    sorted_points = sorted(points, key=lambda p: p[2])  # ordena por altura (z)

    lpr = np.mean(sorted_points[:NLPR], axis=0)
    seeds = [p for p in points if p[2] < lpr[2] + Thseeds]

    return seeds

def estimate_plane(points):
    pts = np.array(points)
    mean = np.mean(pts, axis=0)
    centered = pts - mean

    cov = np.cov(centered.T)
    _, _, vh = np.linalg.svd(cov)
    normal = vh[-1]

    d = -np.dot(normal, mean)

    return (normal, d)

def point_to_plane_distance(point, model):
    normal, d = model

    return abs(np.dot(normal, point) + d) / np.linalg.norm(normal)

def split_into_segments(point_cloud, n_segs):
    points = sorted(point_cloud, key=lambda p: p[0])
    seg_len = len(points) // n_segs

    return [points[i*seg_len:(i+1)*seg_len] for i in range(n_segs)]


# Number of lowest points to use for initial ground seed estimation (LPR)
N_LPR = 20

# Number of iterations for ground plane refinement
N_ITERATIONS = 3

# Height threshold above the LPR to select initial seed points
TH_SEEDS = 0.4

# Distance threshold to classify a point as ground
TH_DISTANCE_FROM_PLANE = 0.2

RADIUS_FOR_NEAREST_NEIGHBORS = 0.5

MIN_POINTS_FOR_CLUSTER = 1

# dataset = Dataset('../datasets/semantic-kitti-data')
# point_cloud, item = dataset[0]

# point_cloud, plane_model = ground_plane_fitting(point_cloud, N_ITERATIONS, N_LPR, TH_SEEDS, TH_DISTANCE_FROM_PLANE)
# normal, d = plane_model

# point_cloud = process_non_ground_clusters(point_cloud, eps=RADIUS_FOR_NEAREST_NEIGHBORS, min_pts=MIN_POINTS_FOR_CLUSTER)

# center = point_cloud[:, :3].mean(axis=0)
# plane = generate_plane_points(normal, d, center)
# point_cloud = np.vstack((point_cloud, plane))

# visualize_clusters(point_cloud, show_plane=True)

In [19]:
def scan_line_run_clustering(point_cloud):
    points = [p for p in point_cloud if p[4] == 0]
    points_index = [i for i, p in enumerate(point_cloud) if p[4] == 0]
    points = np.stack(points)
    scanlines = group_by_scanline(points)
    labels = {}
    label_counter = 1
    label_equiv = {}

    runs_above, label_counter = label_runs(scanlines[0], label_counter)
    
    for i in range(1, len(scanlines)):
        runs_curr, label_counter = label_runs(scanlines[i], label_counter)
        update_labels(runs_curr, runs_above, label_equiv)
        for pt, lbl in runs_above:
            labels[pt] = lbl 
        runs_above = runs_curr

    resolve_label_conflicts(labels, label_equiv)
    points = extract_clusters(labels)
    pt_to_idx = {tuple(points[i]): points_index[i] for i in range(len(points))}

    for pt, idx in pt_to_idx.items():
        point_cloud[idx] = np.array(pt, dtype=point_cloud.dtype)

    return point_cloud

def group_by_scanline(points):
    scanlines = defaultdict(list)
    for p in points:
        scan_id = int(p[5])
        scanlines[scan_id].append(p)
    return [np.array(scanlines[k]) for k in sorted(scanlines.keys())]

def label_runs(scanline, label_counter):
    runs = []
    current_run = []
    for idx, point in enumerate(scanline):
        if not current_run or np.linalg.norm(point - scanline[idx-1]) < RADIUS_FOR_NEAREST_NEIGHBORS:
            current_run.append(point)
        else:
            label = label_counter
            label_counter += 1
            for pt in current_run:
                pt_id = tuple(pt)
                runs.append((pt_id, label))
            current_run = [point]
    if current_run:
        label = label_counter
        label_counter += 1
        if label_counter == 9:
            label_counter += 1
        for pt in current_run:
            pt_id = tuple(pt)
            runs.append((pt_id, label))
    return runs, label_counter

def update_labels(current_runs, previous_runs, label_equiv):
    prev_dict = dict(previous_runs)
    for pt_id, label in current_runs:
        nn_label = find_nearest_label(pt_id, prev_dict)
        if nn_label:
            if label != nn_label:
                label_equiv[label] = nn_label

def find_nearest_label(pt_id, prev_dict):
    pt = np.array(pt_id)
    min_dist = float('inf')
    nearest = None
    for other_pt_id, lbl in prev_dict.items():
        dist = np.linalg.norm(pt[:3] - np.array(other_pt_id)[:3])
        if dist < Thmerge and dist < min_dist:
            min_dist = dist
            nearest = lbl
    return nearest

def resolve_label_conflicts(labels, label_equiv):
    for pt_id, lbl in labels.items():
        while lbl in label_equiv:
            lbl = label_equiv[lbl]
        labels[pt_id] = lbl

def extract_clusters(labels):
    points = []
    for pt_id, label in labels.items():
        p = np.array(pt_id)
        p[4] = label
        points.append(p)
    return np.stack(points)

# Exemplo de ponto: [x, y, z]
def get_scan_id(num_points, N_SCAN=64):
    # # Exemplo fictício baseado em altura ou ângulo vertical
    # return int(point[1] // 1.0)  # ajuste conforme o sensor
    #----------------------------------------------------------------------------
    # inferencia calculando o angulo
    # x, y, z = point[:3]
    # r = np.linalg.norm([x, y, z])
    # vertical_angle = np.arcsin(z / r) * 180 / np.pi  # em graus
    # # Aqui assume-se que o LiDAR tem N_SCAN camadas uniformemente distribuídas entre -24.8 e +2 graus (exemplo Velodyne HDL-64E)
    # angle_min = -24.8
    # angle_max = 2.0
    # scan_id = int(((vertical_angle - angle_min) / (angle_max - angle_min)) * N_SCAN)
    # scan_id = np.clip(scan_id, 0, N_SCAN - 1)  # garante que está no intervalo
    #----------------------------------------------------------------------------
    scan_ids = np.repeat(np.arange(N_SCAN), num_points // N_SCAN)
    return scan_ids



# -------------------------------------------------------------------------
# -------------------------------------------------------------------------
# -------------------------------------------------------------------------
# -------------------------------------------------------------------------
# Number of lowest points to use for initial ground seed estimation (LPR)
N_LPR = 2000

# Number of iterations for ground plane refinement
N_ITERATIONS = 5

# Height threshold above the LPR to select initial seed points
TH_SEEDS = 0.4

# Distance threshold to classify a point as ground
TH_DISTANCE_FROM_PLANE = 0.2

RADIUS_FOR_NEAREST_NEIGHBORS = 0.5

Thmerge = 1.0

MIN_POINTS_FOR_CLUSTER = 1

dataset = Dataset('../datasets/semantic-kitti-data')
point_cloud, item = dataset[0]

point_cloud, normal, d = gpf_refinement(point_cloud, 
                                        number_of_lowest_points=N_LPR, 
                                        threshold_seeds=TH_SEEDS, 
                                        plane_distance_threshold=TH_DISTANCE_FROM_PLANE,
                                        num_iterations=N_ITERATIONS)

# point_cloud = process_non_ground_clusters(point_cloud, eps=RADIUS_FOR_NEAREST_NEIGHBORS, min_pts=MIN_POINTS_FOR_CLUSTER)
point_cloud = scan_line_run_clustering(point_cloud)

center = point_cloud[:, :3].mean(axis=0)
plane = generate_plane_points(normal, d, center)
point_cloud = np.vstack((point_cloud, plane))

visualize_clusters(point_cloud, show_plane=True)

np.save('frame_segmentado_slr_igual.npy', point_cloud)

In [20]:
frame_segmentado_1 = np.load('frame_segmentado_slr_diferente.npy')
frame_segmentado_2 = np.load('frame_segmentado_slr_igual.npy')

visualize_clusters(frame_segmentado_1, show_true_label=True)
visualize_clusters(frame_segmentado_1, show_plane=True)
visualize_clusters(frame_segmentado_2, show_plane=True)

In [21]:
main('./pointclouds/1504941055.292141000.pcd')
main('./pointclouds/1504941060.199916000.pcd')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 1 has size 6

In [ ]:
def run_all_pointclouds(folder_path='./pointclouds'):
    """
    Find and process all .pcd files in the specified folder using the main() function.
    """
    pcd_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pcd')])

    for filename in pcd_files:
        file_path = os.path.join(folder_path, filename)
        print(f"\nProcessing: {filename}")
        try:
            main(file_path)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Run the batch
# run_all_pointclouds('./pointclouds')